In [1]:
import numpy as np
import gc
import timeit


# Concat large arrays 

In [2]:
real_shape = (206928225, 100)
testshape = (int(206928225/100), 100)
testshape
test = True



In [3]:
if test:
    shape = testshape
else:
    shape = real_shape

In [4]:
def fast_concat(se,pe,oe):
    
    assert se.shape == pe.shape, "Error! fast_concat with differing shapes"
    assert se.shape == oe.shape, "Error! fast_concat with differing shapes"
       
    x = np.empty((se.shape[0],se.shape[1]*3),dtype=np.float32)
    x[:,0:100] =se
    x[:,100:200] = pe
    x[:,200:] = oe
    
    return x

In [ ]:
testarray0 = np.zeros(shape)
testarray1= np.zeros(shape)
testarray2 = np.zeros(shape)

In [5]:
try_concat = f"np.concatenate([testarray0 ,testarray1,testarray2],axis=1);gc.collect()"
try_empty = f"fast_concat(testarray0,testarray1,testarray2 );gc.collect()"

In [6]:
timeit.timeit(try_concat, number=3,globals=globals())

5.631752512003004

In [7]:
timeit.timeit(try_empty, number=3,globals=globals())

3.042917001002934

Speedup of about 2x :/

In [8]:
# pytorch model
import torchmetrics
import torch
from torch import nn 


class ClassifierSimple(torch.nn.Module):
    def __init__(self,input_dim=300,hidden_size=64):
        super(ClassifierSimple, self).__init__()
        
        
        self.layers = nn.Sequential(
                # flatten input if necessary
                nn.Flatten(),
                nn.Linear(input_dim,hidden_size),
                nn.ReLU(),
                nn.Linear(hidden_size,1)
        )
        
        self.output_activation = nn.Sigmoid()
                
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
 
        
    
    def forward(self,x):        
        
        return self.layers(x)
    def predict(self,x):
        x.to(self.device)
        
        return self.output_activation(self.layers(x))
    def predict_numpy(self,x):
        x = torch.tensor(x)
        x.to(self.device)
        return self.output_activation(self.layers(x)).detach().cpu().numpy()
        
    


2022-11-07 18:40:26.724282: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-07 18:40:26.724315: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [9]:
from pathlib import Path
import pandas as pd
model = ClassifierSimple()    

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if Path('rdf2vecClassfier.pth').is_file():
    print('found trained model! Loading :)')
    model.load_state_dict(torch.load('rdf2vecClassfier.pth'))
    history = pd.read_csv('log.csv')
    model = model.to(device)
else:
    model = model.to(device)
    model,history = fit_1_1(model,g_train,lambda x: word_vectors[x],3000,entities,metrics = {'acc' :torchmetrics.classification.Accuracy()},graph_eval=g_val,epochs=300)
    model.eval()
    torch.save(model.state_dict(),'rdf2vecClassfier.pth')

found trained model! Loading :)


In [10]:
test = fast_concat(testarray0,testarray1,testarray2 )

# Test Prediction function

In [18]:
timeit.timeit(f"model(torch.tensor(test));gc.collect()", number=100,globals=globals())

132.47281811899666

In [19]:
timeit.timeit(f"model.predict_numpy(test);gc.collect()", number=100,globals=globals())

134.44374255100047

No Difference on cpu ... need to check on gpu